Importacion de GeoJson para mapPlot

In [2]:
import json
import db
import plotly.express as px
from urllib.request import urlopen
import json
import pandas as pd
from pandarallel import pandarallel
from numpy import log as ln

In [3]:
nyc_geojson = './assets/borough_nyc.geojson'
counties = json.load(open(nyc_geojson))

In [ ]:
# ID de borough en GeoJson
d = ['36005', '36047', '36061', '36081', '36085']
name = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens',  'Staten Island']

In [ ]:
borough_map = {}
for feature in counties['features']:
    feature['id'] = feature['properties']['boro_code']
    borough_map[feature['properties']['boro_name']] = feature['id']

borough_id = pd.Series(borough_map)
borough_id = pd.DataFrame(borough_id)
borough_id.reset_index(inplace=True)
borough_id.rename(columns={'index': 'borough', 0:'id_borough'}, inplace=True)



In [ ]:
# data frame de boroughs a imprimir y su escala de color
c = pd.DataFrame()
c['fips'] = d
c['borough'] = name
c['unemp'] = [1,3,6,8,11]

In [4]:
table = db.conn(
    '''
    SELECT *
    FROM taxi_trips;
    '''
)

[2022-07-28 07:56:18] - Iniciando conexion a base de datos...
[2022-07-28 07:56:19] - Conexion Exitosa.
[2022-07-28 07:56:38] - Tabla cargada exitosamente.


Obtenciond de taxi_trip y su transofrmacion

Obtencion de grafica

In [ ]:
# marge DropOff location
taxi_trip_2018 = taxi_trip_2018.merge(df_Taxi_zone, how = 'left', right_on = 'LocationID', left_on = 'IdPULocation')
#taxi_trip_2018.drop(columns= ['IdPULocation', 'LocationID'], inplace=True)
taxi_trip_2018.rename(columns={'Borough' : 'boroughPU'}, inplace=True)

In [ ]:
# marge entre columnas bla bla bla
taxi_trip_2018 = taxi_trip_2018.merge(df_Taxi_zone, how = 'left', right_on = 'LocationID', left_on = 'IdDOLocation')
#taxi_trip_2018.drop(columns= ['IdDOLocation', 'LocationID'], inplace=True)
taxi_trip_2018.rename(columns={'Borough':'boroughDO'}, inplace=True)

In [ ]:
z = (taxi_trip_2018.groupby(['boroughDO', 'boroughPU']).size()).unstack()

In [ ]:
z.fillna(0, inplace=True)
z.drop(columns=['EWR'], inplace=True)
z.drop(['EWR'], axis=0, inplace=True)
z.drop(columns=['Unknown'], axis=1 , inplace = True)
z.drop(['Unknown'], axis=0, inplace=True)

In [ ]:
c.unemp = z.sum().values
c.loc[:,'unemp'] = ln(c.unemp)

In [ ]:
fig = px.choropleth_mapbox(c, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale=px.colors.sequential.Blues,
                           range_color=(0, c.unemp.max()),
                           mapbox_style="carto-positron",
                           zoom=9,
                           center= {"lat": 40.7080747, "lon": -73.9810613},
                           opacity=0.5,
                           labels={'unemp':'trips scale'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()